# 12.23 课后作业： 股票时间序列分析
------

本次作业使用 LSTM 模型对单只股票和多支股票每日收盘价格序列进行分析和预测。

参考资料：
- https://arxiv.org/pdf/1703.04691v5.pdf
- https://arxiv.org/pdf/1805.11317.pdf

In [ ]:
# 导入常用包
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

import os
import time
import random
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt

In [ ]:
# 指定随机种子
seed = 0

random.seed(seed)  # Python random module.
np.random.seed(seed)  # Numpy module.

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

## 1. 单支股票预测

该部分使用简单的单层 LSTM 模型来预测 Google 公司2016年初至2017年末的股价变化。

### 1.1 导入文件

读取股价的数据，并画出股价的图像

In [ ]:
def loadOneFile(file_path):
    data = pd.read_csv(file_path)
    name = data["Name"][0]
    data = data[["Date", "Close"]]
    data.rename(columns={"Close":name}, inplace=True)
    return data

file_path = "time-series-data/GOOGL_2006-01-01_to_2018-01-01.csv"
data = loadOneFile(file_path)
data

In [ ]:
# DataFrame 可视化
def plotDf(df):
    df.plot()
    plt.legend(loc="upper left", ncol=3)
    plt.show()

data.drop(columns="Date", inplace=True)
plotDf(data)

### 1.2 构造 Pytorch 迭代对象

上次作业中，我们直接使用股票价格作为输入数据。但是由于不同公司的股价差别很大，这样的模型不具有泛化能力。在本次作业中，我们使用【日收益率】作为输入的特征。第 t 个 timestep 的日收益率定义为

$$
R_t = \frac{p_{t+1} - p_t}{p_t}
$$

可以忽略日期不连续的问题，把股价视为一个时间序列。具体要求

- 继承 torch.utils.data.Dataset 定义一个可以随机访问和迭代的对象。利用此对象构造训练集、验证集和测试集。

- 构造`Pytorch`中标准的`DataLoader`类。

In [ ]:
n_stocks = 1  # 股票数量
K = 25        # 窗口长度

In [ ]:
class StockDataset(torch.utils.data.Dataset):
    def __init__(self, data, K):
        self.data = data[1:] / data[:-1] - 1
        self.K = K

    def __len__(self):
        return len(self.data) - self.K

    def __getitem__(self, idx):
        # return the `idx` - th object (x, y) in the dataset
        return (self.data[idx : idx + K], 
                self.data[idx + K].unsqueeze(0))


def splitData(data, K):
    data = torch.Tensor(data.to_numpy())
    train_data = StockDataset(data[:-500], K)
    test_data = StockDataset(data[-500:], K)

    train_size = int(0.8 * len(train_data))
    valid_size = len(train_data) - train_size
    train_data, valid_data = torch.utils.data.random_split(train_data, [train_size, valid_size])

    train_iter = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)
    valid_iter = torch.utils.data.DataLoader(valid_data, batch_size=64)
    test_iter = torch.utils.data.DataLoader(test_data, batch_size=64)
    return train_iter, valid_iter, test_iter

train_iter, valid_iter, test_iter = splitData(data, K)

### 1.3 定义模型、损失函数和优化器

使用 LSTM 模型预测股价。由于 LSTM 是一个 seq2seq 的模型，而股票价格预测只需要输出一个值，因此我们只采用 LSTM 的最后一个 cell 的输出。 

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

hidden_size = 16
num_layers = 1

In [ ]:
class MyModel(nn.Module):
    def __init__(self, input_size, hidden, num_layers):
        super(MyModel, self).__init__()
        ### TODO：实现 LSTM 模型



        ### TODO：实现全连接作为后处理



    def forward(self, X):
        ### TODO：实现 forward 部分的计算




net = MyModel(n_stocks, hidden_size, num_layers)
net = net.to(device)
print(net)

关于损失函数，这里我们使用【平均绝对百分比误差（MAPE）】损失函数。该函数的定义如下：

$$
    MAPELoss(pred, label) = \frac{1}{n}\sum_{i}\frac{|pred_i - label_i|}{label_i}
$$

该函数和 MSELoss 的区别主要是：

1. 使用绝对值而不是平方；

2. 使用百分比衡量误差，loss=0%为完美模型，loss>100%表示劣质模型。

MAPE 可以减少不同量级的数据导致的模型误差。股价波动高的公司占据了绝大部分的 MSE，而 MAPE 可以避免这个问题。除此之外，MAPE 还可以直观地给出模型的评价。

In [ ]:
def MAPELoss(preds, labels):
    ### TODO：实现 MAPELoss



loss_fn = MAPELoss
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-4)

### 1.4 模型训练和测试

训练模型。`evaluate_model`用于评估模型`net`在`data_iter`上的表现，给出平均损失；`train`会在`train_iter`上训练模型`net`，优化器选择`optimizer`，并用`valid_iter`进行验证及early stop。

**请你完善以下代码。**代码仅供参考，觉得不方便的地方，可以直接修改。

In [ ]:
from utils import EarlyStop

def evaluate_model(data_iter, net, loss_fn, device=device):
    ###TODO：实现 evaluate


def train(net, train_iter, valid_iter, loss_fn, optimizer, max_epochs=100, early_stop=None, device=device):
    ### TODO：实现 train


In [ ]:
train(net, train_iter, valid_iter, loss_fn, 
      optimizer, max_epochs=300, early_stop=EarlyStop(patience=10))

In [ ]:
print(f"test loss: {evaluate_model(test_iter, net, loss_fn)}")

思考：如果把LSTM的模型中【只用最后一个 cell】改为【使用全部 cell】，应当怎样处理？模型的效果如何？

## 2. 多支股票预测

根据市场有效性理论，股票价格是随机游走（Maurice Kendall 1953），通过股价的**自相关性**预测股价几乎是不可能的。但是，股价可能和**其他时间序列**具有相关性。我们可以利用其他时间序列辅助预测。

本作业中我们使用 31 支股票组成一组向量，重复上述实验。

### 2.1 导入文件

注意不同文件中包含的数据条数不同，需要按日期合并。

In [ ]:
def loadFiles(dir_path):
    data = []
    for i, file in enumerate(os.listdir(dir_path)):
        data.append(loadOneFile(os.path.join(dir_path, file)))
    
    ### TODO：按【日期（"Date"）】合并不同公司股价的 DataFrame


    return data

df = loadFiles("time-series-data/")
df.head()

In [ ]:
# DataFrame可视化
df.drop(columns="Date", inplace=True)
plotDf(df)

### 2.2 构造 Pytorch 迭代对象

In [ ]:
n_stocks = len(df.columns)  # 股票数量
K = 25                      # 窗口长度

In [ ]:
train_iter, valid_iter, test_iter = splitData(df, K)

### 2.3 定义模型、损失函数和优化器

In [ ]:
net = MyModel(n_stocks, hidden_size, num_layers)
net = net.to(device)
print(net)

In [ ]:
loss_fn = MAPELoss
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-4)

### 2.4 模型训练和测试

In [ ]:
train(net, train_iter, valid_iter, loss_fn, 
      optimizer, max_epochs=300, early_stop=EarlyStop(patience=10))

In [ ]:
print("test loss: ", evaluate_model(test_iter, net, loss_fn))

思考：
- 如果使用股价，而不是日收益率，结果会怎样？使用日收益率还有必要做 normalize 吗？
- 如果使用MSE，而不是MAPE，结果会怎样？
- 如果日收益率定义为

  $$
  R_t = \frac{p_{t+1}}{p_t}
  $$

  你有哪些发现？